# Phase 3 - Chewing Sidedness Analysis

## Overview

This notebook analyzes chewing sidedness by comparing the velocity of the midpoint between two nodes (typically node_8 and node_9) relative to a reference plane defined by three head landmark points.

## What This Analysis Does

The analysis determines chewing sidedness by:
1. **Midpoint Calculation**: Computes the midpoint between two tracked nodes (e.g., node_8 and node_9)
2. **Velocity Computation**: Calculates the velocity vector of the midpoint using central differences
3. **Plane Normal Projection**: Projects the velocity vector onto the plane normal to determine sidedness
4. **Classification**: Classifies each frame as "left", "right", or "neutral" based on velocity magnitude and sidedness score

## Methodology

### 1. Midpoint Calculation
- For each frame, compute the midpoint between two tracked nodes (node_8 and node_9)
- The midpoint represents the center of the chewing region

### 2. Velocity Computation
- Calculate velocity using central differences (forward/backward for edge frames)
- Velocity is computed in m/s (assuming coordinates are in meters)
- The velocity vector represents the direction and speed of movement

### 3. Plane Normal Computation
- Compute the plane normal vector from 3 landmark points (typically head landmarks)
- The plane normal defines the reference orientation for sidedness determination

### 4. Sidedness Score
- Project the velocity vector onto the plane normal: `sidedness_score = velocity · normal`
- Positive score = movement in positive normal direction (right side)
- Negative score = movement in negative normal direction (left side)
- Magnitude indicates strength of sidedness

### 5. Classification
- Only classify as left/right during active chewing movements (high velocity)
- Filters out idle movements to focus on actual chewing
- Thresholds:
  - **Velocity threshold**: Minimum velocity to consider as chewing (default: 0.08 m/s)
  - **Sidedness threshold**: Minimum sidedness score for classification (default: 0.02)
- Classification:
  - `velocity < velocity_threshold` → "neutral"
  - `velocity >= velocity_threshold` and `sidedness_score > sidedness_threshold` → "right"
  - `velocity >= velocity_threshold` and `sidedness_score < -sidedness_threshold` → "left"
  - Otherwise → "neutral"

## Outputs

- `chewing_sidedness_analysis.csv`: Frame-by-frame results with positions, velocities, sidedness scores, and classifications
- Summary statistics printed to console including:
  - Total frames and active chewing frames
  - Classification breakdown (left/right/neutral percentages)
  - Average sidedness scores
  - Dominant side


In [ ]:
# === PHASE 3 • CHEWING SIDEDNESS ANALYSIS ===================================
# Analyzes chewing sidedness by comparing velocity of node_8 and node_9 midpoint
# relative to a reference plane defined by three head landmark points.
#
# INPUT:
#   csv_path: Path to CSV file with 3D node data (columns: frame, node, x, y, z, time_s)
#   fps: Frames per second (default 120.0)
#   node_8_name, node_9_name: Names of the two nodes to compute midpoint
#   landmark_nodes: List of 3 node names to use for plane computation
#
# OUTPUT:
#   chewing_sidedness_analysis.csv: Frame-by-frame results
# ============================================================================

# ===================== USER CONFIGURATION =====================
csv_path = r"data/processed/all_nodes_3d_long (1).csv"  # Path to 3D node data CSV
fps = 120.0  # Frames per second
node_8_name = 'node_8'  # First node for midpoint calculation
node_9_name = 'node_9'  # Second node for midpoint calculation
landmark_nodes = ['node_1', 'node_2', 'node_3']  # 3 nodes to define reference plane

# Output directory
out_root = "results"  # Output directory for results

# Classification thresholds
velocity_threshold = 0.08  # Minimum velocity to consider as chewing (m/s)
sidedness_threshold = 0.02  # Minimum sidedness score for classification
# ==============================================================

import pandas as pd
import numpy as np
from pathlib import Path
import os

os.makedirs(out_root, exist_ok=True)


In [ ]:
# ===================== HELPER FUNCTIONS =====================

def load_and_structure_data(csv_path):
    """Load CSV data and convert to structured array format"""
    print("Loading data...")
    df = pd.read_csv(csv_path)
    
    # Get frames and nodes
    frames = df['frame'].unique()
    nodes = df['node'].unique()
    print(f"Frames: {len(frames)}, Nodes: {len(nodes)}")
    print(f"Nodes: {nodes}")
    
    # Prepare data array: frames x nodes x coordinates
    n_frames = len(frames)
    n_nodes = len(nodes)
    X3 = np.full((n_frames, n_nodes, 3), np.nan)
    
    # Fill the array
    for i, node in enumerate(nodes):
        node_data = df[df['node'] == node]
        for _, row in node_data.iterrows():
            frame_idx = int(row['frame'])
            if frame_idx < n_frames:
                X3[frame_idx, i, 0] = row['x'] if pd.notna(row['x']) else np.nan
                X3[frame_idx, i, 1] = row['y'] if pd.notna(row['y']) else np.nan
                X3[frame_idx, i, 2] = row['z'] if pd.notna(row['z']) else np.nan
    
    # Get time array
    times = df[df['node'] == nodes[0]]['time_s'].values
    
    return X3, frames, nodes, times


def find_node_indices(nodes, target_nodes):
    """Find indices of target nodes in the nodes array"""
    indices = []
    for target in target_nodes:
        for i, n in enumerate(nodes):
            if n == target:
                indices.append(i)
                break
    return indices


def compute_plane_normal(X3, frame, landmark_indices):
    """
    Compute plane normal vector from 3 landmark points
    
    Args:
        X3: Data array (frames x nodes x coordinates)
        frame: Current frame index
        landmark_indices: Indices of 3 landmark nodes
    
    Returns:
        normal: Normalized plane normal vector (3D) or None if invalid
    """
    # Extract landmark positions
    p1 = X3[frame, landmark_indices[0], :]
    p2 = X3[frame, landmark_indices[1], :]
    p3 = X3[frame, landmark_indices[2], :]
    
    # Check if all points are valid
    if not (np.all(np.isfinite(p1)) and np.all(np.isfinite(p2)) and np.all(np.isfinite(p3))):
        return None
    
    # Calculate plane normal via cross product
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    norm_mag = np.linalg.norm(normal)
    
    if norm_mag < 1e-10:
        return None
    
    normal = normal / norm_mag  # Normalize
    
    return normal


def calculate_velocity(midpoints, dt, fps):
    """
    Calculate velocity using central differences
    
    Args:
        midpoints: Array of midpoint positions (frames x 3)
        dt: Time step
        fps: Frames per second
    
    Returns:
        velocities: Array of velocity vectors (frames x 3)
    """
    n_frames = len(midpoints)
    velocities = np.full((n_frames, 3), np.nan)
    
    for i in range(n_frames):
        if i == 0:
            # Forward difference for first frame
            if np.all(np.isfinite(midpoints[i])) and np.all(np.isfinite(midpoints[i+1])):
                velocities[i] = (midpoints[i+1] - midpoints[i]) / dt
        elif i == n_frames - 1:
            # Backward difference for last frame
            if np.all(np.isfinite(midpoints[i])) and np.all(np.isfinite(midpoints[i-1])):
                velocities[i] = (midpoints[i] - midpoints[i-1]) / dt
        else:
            # Central difference for middle frames
            if (np.all(np.isfinite(midpoints[i-1])) and 
                np.all(np.isfinite(midpoints[i+1]))):
                velocities[i] = (midpoints[i+1] - midpoints[i-1]) / (2 * dt)
    
    return velocities


def compute_sidedness_score(velocity, plane_normal):
    """
    Compute sidedness by projecting velocity onto plane normal
    
    Args:
        velocity: Velocity vector (3D)
        plane_normal: Plane normal vector (3D)
    
    Returns:
        sidedness_score: Dot product of velocity and normal
    """
    if velocity is None or plane_normal is None:
        return np.nan
    
    return np.dot(velocity, plane_normal)


def classify_sidedness(sidedness_score, velocity_magnitude, velocity_threshold=0.08, sidedness_threshold=0.02):
    """
    Classify sidedness based on score and velocity magnitude
    
    Only classify as left/right during active chewing movements (high velocity).
    This filters out idle movements and focuses on actual chewing.
    
    Args:
        sidedness_score: Score from dot product
        velocity_magnitude: Magnitude of velocity vector
        velocity_threshold: Minimum velocity to consider as chewing (m/s)
        sidedness_threshold: Minimum sidedness score for classification
    
    Returns:
        classification: 'right', 'left', or 'neutral'
    """
    if np.isnan(sidedness_score) or np.isnan(velocity_magnitude):
        return 'neutral'
    
    # First check: if velocity is too low, it's not chewing
    if velocity_magnitude < velocity_threshold:
        return 'neutral'
    
    # Second check: if sidedness score is significant
    if sidedness_score > sidedness_threshold:
        return 'right'
    elif sidedness_score < -sidedness_threshold:
        return 'left'
    else:
        return 'neutral'
# ==============================================================


In [ ]:
# ===================== MAIN ANALYSIS =====================

print("="*60)
print("Chewing Sidedness Analysis")
print("="*60)

dt = 1.0 / fps

# Load data
X3, frames, nodes, times = load_and_structure_data(csv_path)

# Find indices for node_8, node_9, and landmarks
node_8_idx = find_node_indices(nodes, [node_8_name])
node_9_idx = find_node_indices(nodes, [node_9_name])

if not node_8_idx or not node_9_idx:
    print(f"Error: Could not find {node_8_name} or {node_9_name}")
    raise RuntimeError(f"Could not find {node_8_name} or {node_9_name}")

node_8_idx = node_8_idx[0]
node_9_idx = node_9_idx[0]

print(f"{node_8_name} index: {node_8_idx}, {node_9_name} index: {node_9_idx}")

# For plane computation, we need 3 reference points
landmark_indices = find_node_indices(nodes, landmark_nodes)

if len(landmark_indices) != 3:
    print(f"Warning: Could not find all landmarks. Found indices: {landmark_indices}")
    print("Using alternative approach...")
    landmark_indices = [0, 1, 2]  # Use first three nodes as fallback

print(f"Landmark indices: {landmark_indices}")

# Calculate midpoints and velocities
print("\nCalculating midpoints and velocities...")
midpoints = np.full((len(frames), 3), np.nan)
velocities = np.full((len(frames), 3), np.nan)
plane_normals = np.full((len(frames), 3), np.nan)

for i in range(len(frames)):
    # Get positions
    pos_8 = X3[i, node_8_idx, :]
    pos_9 = X3[i, node_9_idx, :]
    
    # Calculate midpoint
    if np.all(np.isfinite(pos_8)) and np.all(np.isfinite(pos_9)):
        midpoints[i] = (pos_8 + pos_9) / 2
    
    # Calculate plane normal
    normal = compute_plane_normal(X3, i, landmark_indices)
    if normal is not None:
        plane_normals[i] = normal

# Calculate velocities
velocities = calculate_velocity(midpoints, dt, fps)

# Calculate sidedness
print("Calculating sidedness scores...")
sidedness_scores = []
sidedness_classifications = []

for i in range(len(frames)):
    vel = velocities[i] if np.all(np.isfinite(velocities[i])) else None
    normal = plane_normals[i] if np.all(np.isfinite(plane_normals[i])) else None
    
    score = compute_sidedness_score(vel, normal)
    velocity_mag = np.linalg.norm(velocities[i]) if np.all(np.isfinite(velocities[i])) else np.nan
    
    sidedness_scores.append(score)
    sidedness_classifications.append(classify_sidedness(score, velocity_mag, velocity_threshold, sidedness_threshold))

# Create output DataFrame
print("Creating output...")
results = []

for i in range(len(frames)):
    pos_8 = X3[i, node_8_idx, :]
    pos_9 = X3[i, node_9_idx, :]
    midpoint = midpoints[i]
    velocity = velocities[i]
    normal = plane_normals[i]
    score = sidedness_scores[i]
    classification = sidedness_classifications[i]
    
    result_row = {
        'frame': frames[i],
        'time_s': times[i],
        'node_8_x': pos_8[0] if np.isfinite(pos_8[0]) else np.nan,
        'node_8_y': pos_8[1] if np.isfinite(pos_8[1]) else np.nan,
        'node_8_z': pos_8[2] if np.isfinite(pos_8[2]) else np.nan,
        'node_9_x': pos_9[0] if np.isfinite(pos_9[0]) else np.nan,
        'node_9_y': pos_9[1] if np.isfinite(pos_9[1]) else np.nan,
        'node_9_z': pos_9[2] if np.isfinite(pos_9[2]) else np.nan,
        'midpoint_x': midpoint[0] if np.isfinite(midpoint[0]) else np.nan,
        'midpoint_y': midpoint[1] if np.isfinite(midpoint[1]) else np.nan,
        'midpoint_z': midpoint[2] if np.isfinite(midpoint[2]) else np.nan,
        'velocity_x': velocity[0] if np.isfinite(velocity[0]) else np.nan,
        'velocity_y': velocity[1] if np.isfinite(velocity[1]) else np.nan,
        'velocity_z': velocity[2] if np.isfinite(velocity[2]) else np.nan,
        'velocity_magnitude': np.linalg.norm(velocity) if np.all(np.isfinite(velocity)) else np.nan,
        'plane_normal_x': normal[0] if np.isfinite(normal[0]) else np.nan,
        'plane_normal_y': normal[1] if np.isfinite(normal[1]) else np.nan,
        'plane_normal_z': normal[2] if np.isfinite(normal[2]) else np.nan,
        'sidedness_score': score,
        'sidedness_classification': classification
    }
    results.append(result_row)

results_df = pd.DataFrame(results)

# Add summary statistics
total = len(sidedness_classifications)
left_count = sum(1 for c in sidedness_classifications if c == 'left')
right_count = sum(1 for c in sidedness_classifications if c == 'right')
neutral_count = sum(1 for c in sidedness_classifications if c == 'neutral')

# Count frames with active chewing (velocity > threshold)
active_chewing_frames = sum(1 for i in range(total) 
                             if np.isfinite(results_df.loc[i, 'velocity_magnitude']) 
                             and results_df.loc[i, 'velocity_magnitude'] >= velocity_threshold)

# Calculate percentages
left_percent = 100 * left_count / total
right_percent = 100 * right_count / total
neutral_percent = 100 * neutral_count / total
active_chewing_percent = 100 * active_chewing_frames / total

# Average sidedness score only for frames with active chewing
active_chewing_rows = results_df[results_df['velocity_magnitude'] >= velocity_threshold]
avg_score_active = np.nanmean(active_chewing_rows['sidedness_score']) if len(active_chewing_rows) > 0 else np.nan
avg_score_all = np.nanmean(sidedness_scores)

dominant_side = 'right' if right_percent > left_percent else 'left' if left_percent > right_percent else 'neutral'

print(f"\n{'='*60}")
print("SUMMARY STATISTICS")
print(f"{'='*60}")
print(f"Total frames: {total}")
print(f"Active chewing frames: {active_chewing_percent:.2f}% ({active_chewing_frames} frames)")
print(f"\nClassification breakdown:")
print(f"  Left sided: {left_percent:.2f}% ({left_count} frames)")
print(f"  Right sided: {right_percent:.2f}% ({right_count} frames)")
print(f"  Neutral: {neutral_percent:.2f}% ({neutral_count} frames)")
print(f"\nAverage sidedness score:")
print(f"  All frames: {avg_score_all:.6f}")
print(f"  Active chewing frames: {avg_score_active:.6f}")
print(f"\nDominant side: {dominant_side}")
print(f"{'='*60}")

# Save results
output_path = Path(out_root) / 'chewing_sidedness_analysis.csv'
output_path.parent.mkdir(exist_ok=True, parents=True)
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to: {output_path}")
print("[DONE]")
